In [1]:
from sagemaker.model_monitor import DataCaptureConfig

# Please fill in the following for enabling data capture
s3_capture_upload_path = f"s3://sagemaker-us-west-1-798223350085/monitoring/"


/Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/31/25 22:33:39] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=189708;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=377668;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/pengzhao/Library/Application Support/sagemaker/config.yaml


In [3]:
import sagemaker
import boto3
# SageMaker session
sess = sagemaker.Session()
# get session bucket name
bucket = sess.default_bucket()
# bucket prefix or the subfolder for everything we produce
prefix = "mlops"
# get sagemaker role
#sagemaker_role = sagemaker.get_execution_role()
sagemaker_role = "arn:aws:iam::798223350085:role/FULLSTACKDS_ROLE1"
# s3 client
s3_client = boto3.client("s3")

region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)


sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client
)

[01/31/25 22:34:35] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=569673;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=902175;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

[01/31/25 22:34:36] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=538152;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=755350;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=157715;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=596416;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

In [4]:
%%time
endpoint_name = "CustomerChurn"
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session
)

# Change parameters as you would like - adjust sampling percentage,
#  chose to capture request or response or both
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
    kms_key_id=None,
    capture_options=["REQUEST", "RESPONSE"],
    csv_content_types=["text/csv"],
    json_content_types=["application/json"],
)

# Now it is time to apply the new configuration and wait for it to be applied
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sess.wait_for_endpoint(endpoint=endpoint_name)

[01/31/25 22:34:51] INFO     Creating endpoint-config with name                                     ]8;id=775447;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=833376;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py#4593\4593]8;;\
                             CustomerChurn-2025-02-01-06-34-51-791                                                 

-------!!CPU times: user 313 ms, sys: 225 ms, total: 538 ms
Wall time: 4min 32s


{'EndpointName': 'CustomerChurn',
 'EndpointArn': 'arn:aws:sagemaker:us-west-1:798223350085:endpoint/CustomerChurn',
 'EndpointConfigName': 'CustomerChurn-2025-02-01-06-34-51-791',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/xgboost:latest',
     'ResolvedImage': '632365934929.dkr.ecr.us-west-1.amazonaws.com/xgboost@sha256:0c8f830ac408e6dee08445fb60392e9c3f05f790a4b3c07ec22327c08f75bcbf',
     'ResolutionTime': datetime.datetime(2025, 1, 31, 22, 34, 53, 162000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 100,
  'DestinationS3Uri': 's3://sagemaker-us-west-1-798223350085/monitoring/'},
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2025, 1, 31, 21, 25, 8, 607000, tzinfo=tzlocal())

In [5]:
##'s3://bucketname/path/to/baseline/data' - Where your validation data is
baseline_data_uri = f"s3://sagemaker-us-west-1-798223350085/ml_deploy/validation/"
##'s3://bucketname/path/to/baseline/data' - Where the results are to be stored in
baseline_results_uri = f"s3://sagemaker-us-west-1-798223350085/monitoring/baseline/results"

print("Baseline data uri: {}".format(baseline_data_uri))
print("Baseline results uri: {}".format(baseline_results_uri))

Baseline data uri: s3://sagemaker-us-west-1-798223350085/ml_deploy/validation/
Baseline results uri: s3://sagemaker-us-west-1-798223350085/monitoring/baseline/results


In [7]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker import get_execution_role
import datetime

#role = get_execution_role(sagemaker_session=sess)
role = sagemaker_role
prefix = "customerchurn"

datetime_stamp = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    base_job_name=f"{prefix}-monitor-{datetime_stamp}",
)

[01/31/25 22:49:29] INFO     Ignoring unnecessary instance type: None.                            ]8;id=422127;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=733580;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/image_uris.py#524\524]8;;\

In [8]:
%%time

monitor_baseline = my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_uri,
    job_name=f"{prefix}-monitor-baseline-{datetime_stamp}",
    wait=True,
)

[01/31/25 22:49:42] INFO     Creating processing-job with name                                      ]8;id=618688;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=568133;file:///Users/pengzhao/opt/anaconda3/envs/mlops/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             customerchurn-monitor-baseline-2025-01-31-224929                                      

.............2025-02-01 06:51:49.234927: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-01 06:51:49.234959: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-02-01 06:51:50.853795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-02-01 06:51:50.853827: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-02-01 06:51:50.853851: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-119-186.us-west-1.compute.internal): /proc/driver/nvidia/version does not exist
2025-02-01 06:51:50.8

In [9]:
from time import gmtime, strftime
import boto3

client = boto3.client("sagemaker")


def get_last_processing_job():
    response = client.list_processing_jobs(
        NameContains=f"{prefix}-monitor-baseline-{datetime_stamp}",
        StatusEquals="Completed",
        SortBy="CreationTime",
        SortOrder="Descending",
        MaxResults=20,
    )
    pprint.pprint(response["ProcessingJobSummaries"][0])
    return response["ProcessingJobSummaries"][0]["ProcessingJobName"]

In [10]:
from sagemaker.processing import ProcessingJob
from sagemaker.estimator import Estimator
from sagemaker.model_monitor.model_monitoring import ModelMonitor
import pprint
my_default_monitor_name = get_last_processing_job()

{'CreationTime': datetime.datetime(2025, 1, 31, 22, 49, 42, 925000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2025, 1, 31, 22, 54, 44, 432000, tzinfo=tzlocal()),
 'ProcessingEndTime': datetime.datetime(2025, 1, 31, 22, 54, 2, tzinfo=tzlocal()),
 'ProcessingJobArn': 'arn:aws:sagemaker:us-west-1:798223350085:processing-job/customerchurn-monitor-baseline-2025-01-31-224929',
 'ProcessingJobName': 'customerchurn-monitor-baseline-2025-01-31-224929',
 'ProcessingJobStatus': 'Completed'}


In [15]:
my_default_monitor_reload = ProcessingJob.from_processing_name(sess, my_default_monitor_name)

response = client.describe_processing_job(ProcessingJobName=my_default_monitor_name)
pprint.pprint(response)

{'AppSpecification': {'ImageUri': '159807026194.dkr.ecr.us-west-2.amazonaws.com/sagemaker-model-monitor-analyzer'},
 'CreationTime': datetime.datetime(2024, 3, 30, 23, 11, 34, 579000, tzinfo=tzlocal()),
 'Environment': {'dataset_format': '{"csv": {"header": false, '
                                   '"output_columns_position": "START"}}',
                 'dataset_source': '/opt/ml/processing/input/baseline_dataset_input',
                 'output_path': '/opt/ml/processing/output',
                 'publish_cloudwatch_metrics': 'Disabled'},
 'ExitMessage': 'Completed: Job completed successfully with no violations.',
 'LastModifiedTime': datetime.datetime(2024, 3, 30, 23, 18, 25, 774000, tzinfo=tzlocal()),
 'ProcessingEndTime': datetime.datetime(2024, 3, 30, 23, 18, 25, 171000, tzinfo=tzlocal()),
 'ProcessingInputs': [{'AppManaged': False,
                       'InputName': 'baseline_dataset_input',
                       'S3Input': {'LocalPath': '/opt/ml/processing/input/baseline_da

In [11]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.approximate_num_distinct_values,numerical_statistics.completeness,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,_c0,Fractional,1710,0,0.508187,869.000000,0.499933,0.000000,1.000000,2,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,_c1,Fractional,1710,0,103.608772,177171.000000,58.049159,1.000000,200.000000,201,1.0,"[{'lower_bound': 1.0, 'upper_bound': 20.9, 'co...",0.64,2048.0,"[[1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0,..."
2,_c2,Fractional,1710,0,229.941520,393200.000000,275.157578,0.000000,1100.000000,12,1.0,"[{'lower_bound': 0.0, 'upper_bound': 110.0, 'c...",0.64,2048.0,"[[0.0, 0.0, 500.0, 0.0, 0.0, 400.0, 0.0, 0.0, ..."
3,_c3,Fractional,1710,0,5.503563,9411.093407,3.316031,0.000215,16.504147,1723,1.0,"[{'lower_bound': 0.0002148017674441505, 'upper...",0.64,2048.0,"[[1.0526538076293468, 1.442509561551946, 7.035..."
4,_c4,Fractional,1710,0,3.498830,5983.000000,1.697865,0.000000,9.000000,10,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.9, 'cou...",0.64,2048.0,"[[2.0, 3.0, 5.0, 3.0, 5.0, 5.0, 4.0, 7.0, 4.0,..."
5,_c5,Fractional,1710,0,5.031373,8603.647048,2.086657,0.005548,11.112419,1671,1.0,"[{'lower_bound': 0.005548397942751747, 'upper_...",0.64,2048.0,"[[0.6993549113167711, 2.2467101421396123, 3.13..."
6,_c6,Fractional,1710,0,3.170760,5422.000000,2.529607,0.000000,12.000000,13,1.0,"[{'lower_bound': 0.0, 'upper_bound': 1.2, 'cou...",0.64,2048.0,"[[2.0, 1.0, 5.0, 5.0, 7.0, 7.0, 8.0, 3.0, 5.0,..."
7,_c7,Fractional,1710,0,4.049233,6924.188944,1.666557,0.024479,10.183378,1748,1.0,"[{'lower_bound': 0.024479438937142103, 'upper_...",0.64,2048.0,"[[5.073754840111169, 5.052209883015463, 5.1068..."
8,_c8,Fractional,1710,0,222.017544,379650.000000,95.951802,0.000000,550.000000,12,1.0,"[{'lower_bound': 0.0, 'upper_bound': 55.0, 'co...",0.64,2048.0,"[[200.0, 150.0, 200.0, 400.0, 300.0, 300.0, 40..."
9,_c9,Fractional,1710,0,5.037326,8613.827033,1.012575,2.024237,8.405644,1566,1.0,"[{'lower_bound': 2.02423709801934, 'upper_boun...",0.64,2048.0,"[[5.823936374928548, 4.227688847116808, 5.5186..."


In [12]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Fractional,1.0,True
1,_c1,Fractional,1.0,True
2,_c2,Fractional,1.0,True
3,_c3,Fractional,1.0,True
4,_c4,Fractional,1.0,True
5,_c5,Fractional,1.0,True
6,_c6,Fractional,1.0,True
7,_c7,Fractional,1.0,True
8,_c8,Fractional,1.0,True
9,_c9,Fractional,1.0,True


In [16]:
sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name='us-west-1')

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Body = "2.0,400.0,0.38571846040122537,2.0,4.177940384158745,0.0,3.745462710628048,250.0,3.699591756294294,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0"
)
print(response['Body'].read().decode('utf-8'))

0.702198326587677


In [17]:
for x in range(20):
    response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Body = "2.0,400.0,0.38571846040122537,2.0,4.177940384158745,0.0,3.745462710628048,250.0,3.699591756294294,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0"
)
    print(response['Body'].read().decode('utf-8'))

0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677
0.702198326587677


In [18]:
import time

MODEL_MONITOR_SCHEDULE_NAME = "mlops-data-quality-schedule"
sm_client = boto3.client("sagemaker")
monitoring_executions = sm_client.list_monitoring_executions(
    MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME
)

# Check if monitoring job has started
while not (monitoring_executions.get("MonitoringExecutionSummaries")):

    # Progress update
    print("Waiting for the data drift model monitoring processing job to start...")

    # Pause for 60 seconds
    time.sleep(60)

    # Get the first monitoring execution details
    monitoring_executions = sm_client.list_monitoring_executions(
        MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME
    )

print("The data drift model monitoring processing job has started.")

Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:16                                                                                   │
│                                                                                                  │
│   13 │   print("Waiting for the data drift model monitoring processing job to start...")         │
│   14 │                                                                                           │
│   15 │   # Pause for 60 seconds                                                                  │
│ ❱ 16 │   time.sleep(60)                                                                          │
│   17 │                                                                                           │
│   18 │   # Get the first monitoring execution details                                            │
│   19 │   monitoring_executions = sm_client.list_monitoring_executions(                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [ ]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()

We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...


In [42]:
latest_execution = mon_executions[-1] # latest execution's index is -1, previous is -2 and so on..


print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")



Latest execution status: Completed
Latest execution result: Completed: Job completed successfully with no violations.


In [43]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

Report Uri: s3://sagemaker-us-west-2-846634201516/reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00


In [44]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: sagemaker-us-west-2-846634201516
Report key: reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00
Found Report Files:
reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00/constraint_violations.json
 reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00/constraints.json
 reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00/statistics.json


In [6]:
import boto3
sm_client = boto3.client("sagemaker")
MODEL_MONITOR_SCHEDULE_NAME = "mlops-data-quality-schedule"

response = sm_client.delete_monitoring_schedule(
    MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME
)


In [7]:
endpoint_name = "CustomerChurn"
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c9939153-4670-4f91-a482-bdafbe4886e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c9939153-4670-4f91-a482-bdafbe4886e1',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 31 Mar 2024 03:29:38 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}